In [1]:
import os


In [2]:
import pandas as pd
import time
import pickle

In [3]:
import matgl
from matgl.ext.ase import PESCalculator, MolecularDynamics, Relaxer

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from disalloy.cellfunc import jitter_ia, jitter_ic, compute_distortion
from disalloy.grid_run import run_packet

In [5]:
pot = matgl.load_model("M3GNet-MP-2021.2.8-PES")

/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/usr/local/lib/python3.9/site-packages/matgl/apps/pes.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_std", torch.tensor(da

In [6]:
relaxer_2021_2_8_PES = Relaxer(pot)

In [7]:
with open('AlCoCrCuFeMnNiV_5_10.pkl', 'rb') as f:
    structure_DB = pickle.load(f)

In [8]:
def relaxer_direct_2021_2_8_PES(structure):
    return relaxer_2021_2_8_PES.relax(structure)
def relaxer_jitter_a_01_2021_2_8_PES(structure, ia = 0.01):
    structure = jitter_ia(structure, ia)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)
def relaxer_jitter_c_005_2021_2_8_PES(structure, ic = 0.01):
    structure = jitter_ic(structure, ic)
    return relaxer_2021_2_8_PES.relax(structure)#, fmax = 0.001)

## creating a dabase 'df_calc' and running the direct relaxation in M3GNet model

In [9]:
df = structure_DB.copy()
if os.path.exists('df_calc.pkl'):
    df = pd.read_pickle('df_calc.pkl')

In [10]:
relaxer_dic_2021_2_8_PES = {'calculator_label': 'relaxer_dic_2021_2_8_PES',
                            'relaxer': relaxer_direct_2021_2_8_PES}

In [11]:
print(df.attrs)

{'number_of_structures': {1: 8, 2: 56, 3: 336, 4: 4046, 5: 15904, 6: 17080, 7: 23968, 8: 128540, 9: 140000, 10: 180000}, 'is_exhaustive': {1: True, 2: True, 3: True, 4: True, 5: True, 6: False, 7: False, 8: False, 9: False, 10: False}}


In [12]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 10, 1)
print('{}s'.format(time.time() - ti))

2024-09-03 13:26:09 - starting packet run: attempting 10 runs of size 1

2024-09-03 13:26:56 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 13:26:56 - Succesfuly ran packet of 8 runs in 34.679503440856934s

47.16540312767029s


In [13]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 2)
print('{}s'.format(time.time() - ti))

2024-09-03 13:26:56 - starting packet run: attempting 100 runs of size 2

2024-09-03 13:33:55 - index of ran strctures:
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]

2024-09-03 13:33:55 - Succesfuly ran packet of 56 runs in 402.5463480949402s

419.63988757133484s


In [14]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_2021_2_8_PES, 100, 3)
print('{}s'.format(time.time() - ti))

2024-09-03 13:33:55 - starting packet run: attempting 100 runs of size 3

2024-09-03 13:48:28 - index of ran strctures:
 [188, 367, 229, 296, 219, 237, 242, 72, 83, 251, 232, 377, 214, 102, 185, 380, 158, 213, 277, 368, 373, 335, 120, 392, 173, 357, 297, 122, 332, 314, 77, 351, 157, 366, 248, 195, 66, 218, 259, 101, 97, 302, 276, 162, 343, 94, 132, 172, 247, 365, 216, 149, 210, 279, 175, 84, 73, 362, 85, 386, 328, 65, 321, 215, 327, 320, 341, 258, 146, 379, 138, 364, 323, 179, 352, 209, 108, 96, 286, 136, 393, 389, 169, 369, 344, 396, 272, 382, 395, 111, 180, 99, 191, 155, 388, 64, 249, 129, 197, 196]

2024-09-03 13:48:28 - Succesfuly ran packet of 100 runs in 857.2818284034729s

872.555344581604s


In [15]:
df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()]

,init_structure,size,relaxer_dic_2021_2_8_PES
0,"(Atom('Al', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Al', [0.0, 0.0, 0.0], index=0)), -3.70..."
1,"(Atom('Co', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Co', [0.0, 0.0, 0.0], index=0)), -7.10..."
2,"(Atom('Cr', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Cr', [0.0, 0.0, 0.0], index=0)), -9.28..."
3,"(Atom('Cu', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Cu', [0.0, 0.0, 0.0], index=0)), -4.09..."
4,"(Atom('Fe', [0.0, 0.0, 0.0], index=0))",1,"((Atom('Fe', [0.0, 0.0, 0.0], index=0)), -8.39..."
...,...,...,...
389,"(Atom('Fe', [0.0, 0.0, 0.0], index=0), Atom('M...",3,"((Atom('Fe', [-5.629169387890962e-07, -1.68769..."
392,"(Atom('Fe', [0.0, 0.0, 0.0], index=0), Atom('V...",3,"((Atom('Fe', [-1.293493491668783e-06, -5.47112..."
393,"(Atom('Mn', [0.0, 0.0, 0.0], index=0), Atom('M...",3,"((Atom('Mn', [-5.217575941290946e-06, -7.41172..."
395,"(Atom('Mn', [0.0, 0.0, 0.0], index=0), Atom('N...",3,"((Atom('Mn', [3.430101454731041e-07, 7.8414824..."


## now we run with another relaxer (using cell relaxer), but we calculate the same indeces previously calculated

In [16]:
relaxer_dic_jitter_c_005_2021_2_8_PES = {
    'calculator_label': 'relaxer_dic_jitter_c_005_2021_2_8_PES',
    'relaxer': relaxer_jitter_c_005_2021_2_8_PES
}

In [17]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 10, 1, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 13:48:30 - starting packet run: attempting 10 runs of size 1

2024-09-03 13:49:42 - index of ran strctures:
 [0, 1, 2, 3, 4, 5, 6, 7]

2024-09-03 13:49:42 - Succesfuly ran packet of 8 runs in 52.78730225563049s

72.41958045959473s


In [18]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 100, 2, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 13:49:42 - starting packet run: attempting 100 runs of size 2

2024-09-03 13:56:58 - index of ran strctures:
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]

2024-09-03 13:56:58 - Succesfuly ran packet of 56 runs in 421.8365819454193s

435.74034214019775s


In [19]:
ti = time.time()
run_packet('df_calc', df, relaxer_dic_jitter_c_005_2021_2_8_PES, 100, 3, 
           copy_calculated=relaxer_dic_2021_2_8_PES['calculator_label'])
print('{}s'.format(time.time() - ti))

2024-09-03 13:56:58 - starting packet run: attempting 100 runs of size 3

2024-09-03 14:13:30 - index of ran strctures:
 [64, 65, 66, 72, 73, 77, 83, 84, 85, 94, 96, 97, 99, 101, 102, 108, 111, 120, 122, 129, 132, 136, 138, 146, 149, 155, 157, 158, 162, 169, 172, 173, 175, 179, 180, 185, 188, 191, 195, 196, 197, 209, 210, 213, 214, 215, 216, 218, 219, 229, 232, 237, 242, 247, 248, 249, 251, 258, 259, 272, 276, 277, 279, 286, 296, 297, 302, 314, 320, 321, 323, 327, 328, 332, 335, 341, 343, 344, 351, 352, 357, 362, 364, 365, 366, 367, 368, 369, 373, 377, 379, 380, 382, 386, 388, 389, 392, 393, 395, 396]

2024-09-03 14:13:30 - Succesfuly ran packet of 100 runs in 970.0003299713135s

991.5492444038391s


In [20]:
calculated_index = df[df[relaxer_dic_2021_2_8_PES['calculator_label']].notnull()].index


In [22]:
for i, j, k  in zip(df.loc[calculated_index, 'init_structure'], 
                df.loc[calculated_index, 'relaxer_dic_2021_2_8_PES'].array, 
             df.loc[calculated_index, 'relaxer_dic_jitter_c_005_2021_2_8_PES'].array):
    print(compute_distortion(i, j[0]))
    print(compute_distortion(i, k[0]))
    print()

1.5728799997364826e-06
0.005545606434794589

3.4711914702793634e-06
0.015701104316383625

1.869416926911698e-05
0.2707021760615836

6.591054889187993e-06
0.00643418721709413

2.8925424054246692e-05
0.01416115962905737

1.5884375891685432e-05
0.01235419998235029

2.5211789102510017e-06
0.00877251349686312

2.0687255793244462e-05
0.27333987170537427

0.16736472938249838
0.155848814406169

0.0033938860022789045
0.002464174431039534

0.08171670991394009
0.09069547871450262

0.05756732034573136
0.05766670759127381

0.04173950421919971
0.042212950800984014

0.1695639041165874
0.16962245469191053

0.022529365287150514
0.022958524331655534

0.001232368835267994
0.010668903679314164

0.01929429646574994
0.019765368768550673

0.005782588211973358
0.007032192902431949

0.007784802038774972
0.011444225561368428

0.010122801532866276
0.010591934981025074

0.03541982092211414
0.03608210056858039

0.048726994810268084
0.04865296469488417

0.013279712043930338
0.01319264483341809

0.03393364884871949
